# Capítulo 6 — A Estabilidade de Sistemas Lineares com Realimentação

**Aluno(a):** Geovana Stefani Lopes Bezerra  
**Disciplina:** Controle e Automação  
**Professor:** Moacy Pereira da Silva  
**Semestre:** 2025.2  

Este notebook faz parte do estudo dirigido da disciplina de Controle e Automação com foco em aplicações de Indústria 4.0.  
O Capítulo 6 de *Sistemas de Controle Moderno* (Dorf & Bishop) trata da estabilidade de sistemas lineares em malha aberta e fechada, apresentando critérios clássicos (Routh–Hurwitz, local de raízes, Nyquist) e discutindo relações entre polos, margens de estabilidade e projeto de compensadores.


## 1. Introdução

A estabilidade é um conceito central em teoria de controle: diz respeito ao comportamento assintótico das soluções do sistema quando perturbado ou quando submetido a entradas. Dorf & Bishop no Capítulo 6 enfatizam a distinção entre estabilidade interna (dos polos) e estabilidade em malha fechada, além de apresentar métodos práticos para avaliar estabilidade sem necessariamente calcular todos os polos explicitamente. Este notebook segue a organização do capítulo: primeiro os conceitos, depois critérios clássicos (Routh, Nyquist, Lugar das Raízes), em seguida demonstrações computacionais e exemplos comentados.

## 2. Conceitos fundamentais de estabilidade

- **Estabilidade assintótica (linear, tempo-invariante):** todos os polos do sistema no semiplano esquerdo (Re(s) < 0).  
- **Estabilidade marginal:** polos no eixo imaginário (Re(s)=0) sem multiplicidade crítica; comportamento oscilatório sustentado.  
- **Instabilidade:** existe pelo menos um polo com Re(s) > 0.  
- **Estabilidade interna vs. de laço fechado:** estabilidade de malha fechada depende da função de transferência em laço fechado T(s)=L(s)/(1+L(s)) com L(s)=C(s)G(s).  

O capítulo explora como propriedades do polinômio característico (1+L(s)=0) se relacionam com estabilidade, e como métodos algébricos e gráficos permitem avaliar-a sem calcular raízes explicitamente.

## 3. Critério de Routh–Hurwitz

### 3.1 Intuição e objetivo
O critério de Routh–Hurwitz fornece uma forma algébrica de determinar quantas raízes de um polinômio têm parte real positiva, sem resolvê-las explicitamente. A construção da tabela de Routh revela sinais e mudanças de sinal que indicam raízes no semiplano direito. Dorf & Bishop apresentam exemplos e casos especiais (linhas nulas, presença de raízes no eixo jω) e mostram procedimentos para lidar com esses casos.

### 3.2 Procedimento
Dado o polinômio característico a_0 s^n + a_1 s^{n-1} + ... + a_n, constrói-se a tabela de Routh com filas contendo combinações lineares dos coeficientes. O número de mudanças de sinal na primeira coluna da tabela indica o número de raízes com Re(s)>0. Casos especiais exigem substituições (ε) ou decomposição por fatores de (s^2 + ω^2) quando aparecem linhas nulas.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from control import tf, feedback, step_response, pole, root_locus, bode, margin
plt.rcParams['figure.figsize'] = (10,4)

def routh_table(coeffs):
    """Compute Routh-Hurwitz table for polynomial with coefficients coeffs (highest power first).
    Returns the Routh table as a 2D numpy array."""
    coeffs = [float(c) for c in coeffs]
    n = len(coeffs) - 1
    cols = int(np.ceil((n+1)/2))
    table = np.zeros((n+1, cols))
    table[0, :len(coeffs[0::2])] = coeffs[0::2]
    table[1, :len(coeffs[1::2])] = coeffs[1::2]
    for i in range(2, n+1):
        for j in range(0, cols-1):
            a = table[i-2,0]
            b = table[i-2,j+1]
            c = table[i-1,0]
            d = table[i-1,j+1]
            if c == 0:
                val = 0.0
            else:
                val = (c*a - table[i-2,j+1]*table[i-1,0]) / c
            table[i,j] = val
    return table

def routh_sign_changes(coeffs):
    T = routh_table(coeffs)
    first_col = T[:,0]
    signs = np.sign(first_col)
    signs_nz = signs[signs!=0]
    changes = np.sum(signs_nz[:-1] * signs_nz[1:] < 0)
    return int(changes), T

coeffs_example = [1, 2, 3, 4]
changes, T = routh_sign_changes(coeffs_example)
print('Mudanças de sinal (número de raízes no RHP):', changes)
print('Tabela de Caminho:\n', T)


## 4. Critério de Nyquist

### 4.1 Ideia principal
O critério de Nyquist relaciona o número de zeros do polinômio característico no semiplano direito com a variação do ângulo da função L(jω) ao percorrer a curva Nyquist. Em termos práticos, conta-se encirclements do ponto -1 no plano complexo pelo diagrama de Nyquist de L(jω) para determinar estabilidade de malha fechada. Dorf & Bishop apresentam a formulação clássica (N = Z - P), onde P é o número de polos de L(s) no semiplano direito e Z o número de zeros do denominador de 1+L(s) no semiplano direito.

### 4.2 Interpretação geométrica e passos práticos
Construir o diagrama de Nyquist (traçar L(jω) para ω de 0 a ∞ e aplicar simetria). Contar o número de voltas no sentido anti-horário em torno de -1. Pode-se usar o diagrama de Bode para inferir margens e comportamento sem traçar o Nyquist completo.


In [ ]:
from control import nyquist_plot

G = tf([1],[1, -1, 2])
C = tf([1],[1])
L = C*G
plt.figure()
nyquist_plot(L, omega=np.logspace(-2, 2, 500))
plt.title('Diagrama de Nyquist exemplo')
plt.grid(True)


## 5. Lugar das raízes (Root Locus)

### 5.1 Propósito e uso
O lugar das raízes mostra como as raízes do polinômio característico (polos em malha fechada) variam quando um parâmetro (tipicamente ganho K) é alterado. É uma ferramenta gráfica fundamental para projetar controladores e garantir estabilidade enquanto se satisfaz especificações de desempenho.

### 5.2 Regras básicas e interpretação
- Polos e zeros de L(s) determinam ramos do lugar; ramos partem dos polos de L(s) e vão em direção aos zeros enquanto K aumenta.  
- Asporções de ramos ao eixo real, ângulos de entrada/saída nos polos/zeros, e pontos de ruptura são usados para traçar o locus.  
- Dorf & Bishop mostram exemplos de como usar o root locus para escolher K que garanta amortecimento desejado (ζ) ou que posicione polos em locais específicos.


In [ ]:
# Exemplo: planta de segunda ordem com ganho K em série
G = tf([1],[1, 3, 2])  # 1/(s^2 + 3s + 2)
plt.figure()
rlist, klist = root_locus(G, Plot=True, grid=True)
plt.title('Lugar das raízes - exemplo')


## 6. Ferramentas numéricas e exemplos práticos

Implementaremos exemplos práticos que ilustram o uso de Routh, Nyquist e Lugar das Raízes na avaliação da estabilidade de malha fechada. Para cada exemplo, calcularemos polos, margens e verificaremos a consistência entre métodos.

In [ ]:
# Exemplo de conjunto
G = tf([1],[1, 2, 1])  # 1/(s^2 + 2s +1)
C = tf([2],[1])       # ganho K=2
L = C*G
T = feedback(L, 1)
poles_open = pole(G)
poles_closed = pole(T)
gm, pm, sm, wg, wp, ws = margin(L)
print('Polos malha aberta:', poles_open)
print('Polos malha fechada:', poles_closed)
print('Margem de ganho:', gm)
print('Margem de fase (deg):', pm)


## 7. Discussão dos resultados

Nesta seção comparamos os métodos e discutimos vantagens/práticas:  
- **Routh** é algébrico e adequado para conta manual e verificação rápida; pode ser sensível a casos especiais que exigem tratamentos (linhas nulas).  
- **Nyquist** traz visão geométrica e é ótima para lidar com sistemas com polos à direita e atrasos, mas requer análise cuidadosa de contornos.  
- **Root locus** é excelente para projeto de ganho e compreensão de movimento dos polos com mudança de K.  
Ao aplicar estas ferramentas em conjunto, obtém-se robustez na análise: verifique consistência entre número de polos em RHP (Routh) e encirclements em Nyquist.

## 8. Relação com Indústria 4.0

A estabilidade é requisito crítico em sistemas industriais. Na Indústria 4.0, além de projetar controladores estáveis, é necessário monitorar estabilidade em tempo real e integrar detecção automática de degradação:
- **Monitoração em tempo real:** calcular margens e detectar aproximação de instabilidade via métricas em tempo real e alertas no PIMS.  
- **Digital Twin:** simular alterações de parâmetros e validar estabilidade em gêmeos digitais antes de aplicar em campo.  
- **Retuning automático e aprendizagem:** técnicas de identificação em linha podem reestimular modelos e retunar controladores quando a estabilidade é ameaçada por mudanças de processo.
